# I. Import Module

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from haversine import haversine
import haversine as hs 
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

#
import re


# II.Load and preprocess data

In [17]:
submission = pd.read_csv('./data/sample_submission.csv')
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

# train.drop(['ID_LAT_LON_YEAR_WEEK'], axis=1, inplace=True)
# test.drop(['ID_LAT_LON_YEAR_WEEK'], axis=1, inplace=True)

In [18]:
train = train.drop(train.columns[0], axis=1)
train

,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,SulphurDioxide_sensor_zenith_angle,SulphurDioxide_solar_azimuth_angle,SulphurDioxide_solar_zenith_angle,SulphurDioxide_SO2_column_number_density_15km,CarbonMonoxide_CO_column_number_density,CarbonMonoxide_H2O_column_number_density,CarbonMonoxide_cloud_height,CarbonMonoxide_sensor_altitude,CarbonMonoxide_sensor_azimuth_angle,CarbonMonoxide_sensor_zenith_angle,CarbonMonoxide_solar_azimuth_angle,CarbonMonoxide_solar_zenith_angle,NitrogenDioxide_NO2_column_number_density,NitrogenDioxide_tropospheric_NO2_column_number_density,NitrogenDioxide_stratospheric_NO2_column_number_density,NitrogenDioxide_NO2_slant_column_number_density,NitrogenDioxide_tropopause_pressure,NitrogenDioxide_absorbing_aerosol_index,NitrogenDioxide_cloud_fraction,NitrogenDioxide_sensor_altitude,NitrogenDioxide_sensor_azimuth_angle,NitrogenDioxide_sensor_zenith_angle,NitrogenDioxide_solar_azimuth_angle,NitrogenDioxide_solar_zenith_angle,Formaldehyde_tropospheric_HCHO_column_number_density,Formaldehyde_tropospheric_HCHO_column_number_density_amf,Formaldehyde_HCHO_slant_column_number_density,Formaldehyde_cloud_fraction,Formaldehyde_solar_zenith_angle,Formaldehyde_solar_azimuth_angle,Formaldehyde_sensor_zenith_angle,Formaldehyde_sensor_azimuth_angle,UvAerosolIndex_absorbing_aerosol_index,UvAerosolIndex_sensor_altitude,UvAerosolIndex_sensor_azimuth_angle,UvAerosolIndex_sensor_zenith_angle,UvAerosolIndex_solar_azimuth_angle,UvAerosolIndex_solar_zenith_angle,Ozone_O3_column_number_density,Ozone_O3_column_number_density_amf,Ozone_O3_slant_column_number_density,Ozone_O3_effective_temperature,Ozone_cloud_fraction,Ozone_sensor_azimuth_angle,Ozone_sensor_zenith_angle,Ozone_solar_azimuth_angle,Ozone_solar_zenith_angle,UvAerosolLayerHeight_aerosol_height,UvAerosolLayerHeight_aerosol_pressure,UvAerosolLayerHeight_aerosol_optical_depth,UvAerosolLayerHeight_sensor_zenith_angle,UvAerosolLayerHeight_sensor_azimuth_angle,UvAerosolLayerHeight_solar_azimuth_angle,UvAerosolLayerHeight_solar_zenith_angle,Cloud_cloud_fraction,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,-0.510,29.290,2019,0,-0.000108,0.603019,-0.000065,0.255668,-98.593887,50.843559,-130.050797,35.874496,-0.000027,0.035370,1589.024536,4061.098145,829530.500000,71.111977,52.775928,-149.875565,25.965214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000117,0.863230,0.000038,0.255668,35.874496,-130.050797,50.843559,-98.593887,-1.280761,829864.546875,-12.628979,35.632416,-138.786446,30.752128,0.115927,2.506609,0.295663,225.731144,0.595473,-12.628979,35.632416,-138.786446,30.752128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.595473,53534.732422,3664.436218,61085.809570,2615.120483,15.568533,0.272292,-12.628986,35.632416,-138.786423,30.752140,3.750994
1,-0.510,29.290,2019,1,0.000021,0.728214,0.000014,0.130988,16.592861,39.137194,-140.874435,28.965133,0.000012,0.036526,1772.574405,1869.040414,829787.287130,-1.019594,38.982368,-140.158048,29.562000,0.000047,1.639765e-05,0.000030,0.000093,7311.869141,-1.935386,0.067038,829859.960368,5.471037,35.265195,-138.343908,30.054262,0.000170,1.172826,0.000143,0.200754,29.071781,-141.814827,43.050213,4.678839,-1.548119,829747.856973,16.152492,43.485327,-142.786141,28.573627,0.116775,2.657704,0.315733,226.172170,0.175166,24.464335,42.596541,-143.097868,28.213655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.213608,63790.296241,3651.190311,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-145.183930,27.251779,4.025176
2,-0.510,29.290,2019,2,0.000514,0.748199,0.000385,0.110018,72.795837,52.868816,-150.191757,23.206415,0.000154,0.035338,2703.236800,2809.138386,829883.8

In [19]:
# fill nan by mean column
train_filled = train.fillna(train.mode().iloc[0])
train_filled


,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,SulphurDioxide_sensor_zenith_angle,SulphurDioxide_solar_azimuth_angle,SulphurDioxide_solar_zenith_angle,SulphurDioxide_SO2_column_number_density_15km,CarbonMonoxide_CO_column_number_density,CarbonMonoxide_H2O_column_number_density,CarbonMonoxide_cloud_height,CarbonMonoxide_sensor_altitude,CarbonMonoxide_sensor_azimuth_angle,CarbonMonoxide_sensor_zenith_angle,CarbonMonoxide_solar_azimuth_angle,CarbonMonoxide_solar_zenith_angle,NitrogenDioxide_NO2_column_number_density,NitrogenDioxide_tropospheric_NO2_column_number_density,NitrogenDioxide_stratospheric_NO2_column_number_density,NitrogenDioxide_NO2_slant_column_number_density,NitrogenDioxide_tropopause_pressure,NitrogenDioxide_absorbing_aerosol_index,NitrogenDioxide_cloud_fraction,NitrogenDioxide_sensor_altitude,NitrogenDioxide_sensor_azimuth_angle,NitrogenDioxide_sensor_zenith_angle,NitrogenDioxide_solar_azimuth_angle,NitrogenDioxide_solar_zenith_angle,Formaldehyde_tropospheric_HCHO_column_number_density,Formaldehyde_tropospheric_HCHO_column_number_density_amf,Formaldehyde_HCHO_slant_column_number_density,Formaldehyde_cloud_fraction,Formaldehyde_solar_zenith_angle,Formaldehyde_solar_azimuth_angle,Formaldehyde_sensor_zenith_angle,Formaldehyde_sensor_azimuth_angle,UvAerosolIndex_absorbing_aerosol_index,UvAerosolIndex_sensor_altitude,UvAerosolIndex_sensor_azimuth_angle,UvAerosolIndex_sensor_zenith_angle,UvAerosolIndex_solar_azimuth_angle,UvAerosolIndex_solar_zenith_angle,Ozone_O3_column_number_density,Ozone_O3_column_number_density_amf,Ozone_O3_slant_column_number_density,Ozone_O3_effective_temperature,Ozone_cloud_fraction,Ozone_sensor_azimuth_angle,Ozone_sensor_zenith_angle,Ozone_solar_azimuth_angle,Ozone_solar_zenith_angle,UvAerosolLayerHeight_aerosol_height,UvAerosolLayerHeight_aerosol_pressure,UvAerosolLayerHeight_aerosol_optical_depth,UvAerosolLayerHeight_sensor_zenith_angle,UvAerosolLayerHeight_sensor_azimuth_angle,UvAerosolLayerHeight_solar_azimuth_angle,UvAerosolLayerHeight_solar_zenith_angle,Cloud_cloud_fraction,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,-0.510,29.290,2019,0,-0.000108,0.603019,-0.000065,0.255668,-98.593887,50.843559,-130.050797,35.874496,-0.000027,0.035370,1589.024536,4061.098145,829530.500000,71.111977,52.775928,-149.875565,25.965214,0.000024,-2.484413e-06,0.000026,0.000074,7311.869141,-2.138135,0.000000,830284.250000,-98.480850,6.098547,-140.801315,24.392952,0.000117,0.863230,0.000038,0.255668,35.874496,-130.050797,50.843559,-98.593887,-1.280761,829864.546875,-12.628979,35.632416,-138.786446,30.752128,0.115927,2.506609,0.295663,225.731144,0.595473,-12.628979,35.632416,-138.786446,30.752128,1164.338745,10920.485315,-0.377114,57.287567,-98.267899,-106.600899,26.892511,0.595473,53534.732422,3664.436218,61085.809570,2615.120483,15.568533,0.272292,-12.628986,35.632416,-138.786423,30.752140,3.750994
1,-0.510,29.290,2019,1,0.000021,0.728214,0.000014,0.130988,16.592861,39.137194,-140.874435,28.965133,0.000012,0.036526,1772.574405,1869.040414,829787.287130,-1.019594,38.982368,-140.158048,29.562000,0.000047,1.639765e-05,0.000030,0.000093,7311.869141,-1.935386,0.067038,829859.960368,5.471037,35.265195,-138.343908,30.054262,0.000170,1.172826,0.000143,0.200754,29.071781,-141.814827,43.050213,4.678839,-1.548119,829747.856973,16.152492,43.485327,-142.786141,28.573627,0.116775,2.657704,0.315733,226.172170,0.175166,24.464335,42.596541,-143.097868,28.213655,1164.338745,10920.485315,-0.377114,57.287567,-98.267899,-106.600899,26.892511,0.213608,63790.296241,3651.190311,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-1

In [20]:
X = train_filled.drop(['emission'], axis=1)
y = train_filled['emission']
X.shape,y.shape

((79023, 74), (79023,))

In [21]:
def feature_scalar(df):
    # scalar
    sc = StandardScaler()
    for i in df.columns:
        df[i] = sc.fit_transform(df[i].values.reshape(-1,1))
    return df

X_scaled = feature_scalar(X)

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_scaled,y,test_size=.2,random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape


((63218, 74), (15805, 74), (63218,), (15805,))

In [66]:
from tensorflow.keras.optimizers.legacy import Adam

lr = 0.002
weight_decay = 1e-5
optimizer = Adam(learning_rate=lr,decay=weight_decay )

In [67]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_val)
y_test = np.array(y_val)

In [70]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Input

# Create a Sequential model
model = keras.Sequential()

# Add the input layer with 74 input neurons
# model.add(Input(shape=(74,)))

model.add(Dense(1280,input_shape = (74,),activation='relu'))
model.add(Dense(2560,activation='relu'))
model.add(Dense(1280,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='linear'))  # Regression output


# Compile the model
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae','mape'])  # Use appropriate loss function for regression

# Print the model summary
model.summary()


Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_53 (Dense)            (None, 1280)              96000     
                                                                 
 dense_54 (Dense)            (None, 2560)              3279360   
                                                                 
 dense_55 (Dense)            (None, 1280)              3278080   
                                                                 
 dense_56 (Dense)            (None, 256)               327936    
                                                                 
 dense_57 (Dense)            (None, 1)                 257       
                                                                 
Total params: 6981633 (26.63 MB)
Trainable params: 6981633 (26.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [71]:
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))


Epoch 1/100
603/988 [=================>............] - ETA: 44s - loss: 22022.1895 - mae: 70.6087 - mape: 2042084992.0000

KeyboardInterrupt: 

In [7]:
import pandas as pd 
df = pd.read_csv('./data_preprocess/df_preprocess.csv', index_col=0)
df.head()

,ID_-0.510_29.290,ID_-0.528_29.472,ID_-0.547_29.653,ID_-0.569_30.031,ID_-0.598_29.102,ID_-0.604_29.896,ID_-0.615_30.885,ID_-0.627_29.773,ID_-0.637_30.763,ID_-0.653_30.447,...,ID_-3.095_29.505,ID_-3.099_29.601,ID_-3.133_29.467,ID_-3.136_30.364,ID_-3.138_30.662,ID_-3.153_30.347,ID_-3.161_28.839,ID_-3.174_29.926,ID_-3.287_29.713,ID_-3.299_30.301
2019_0,3.750994,3.857115,0.500023,100.382980,0.076542,64.49460,76.600110,101.643295,40.153328,28.108765,...,5.375411,69.82686,105.982190,14.774838,23.184301,15.069341,0.106631,41.193130,39.696404,26.111166
2019_1,4.025176,4.213432,0.529315,106.375980,0.081001,68.42294,82.630714,109.828674,41.531597,29.032100,...,5.420106,74.35756,113.068474,15.337234,24.404161,16.217907,0.112878,43.877285,43.529408,28.045977
2019_2,4.231381,4.451489,0.554183,110.766335,0.084788,71.01155,86.668290,115.189064,42.774483,29.870031,...,5.327792,77.72222,118.279810,15.730142,25.097378,16.811665,0.118181,45.760826,45.993286,29.293518
2019_3,4.305286,4.553981,0.561553,113.097940,0.085920,72.57178,88.741880,117.846870,43.542730,30.415354,...,5.422680,79.23992,120.618580,15.979696,25.591475,17.231894,0.119753,46.675327,47.163128,29.950650
2019_4,4.347317,4.676066,0.559731,114.985886,0.085650,73.83461,91.137010,120.702270,43.739470,30.567698,...,5.316918,80.32427,122.382960,15.905249,25.756530,17.642060,0.119364,47.317184,48.742470,30.572742


In [13]:
df.columns[0]

'ID_-0.510_29.290'

In [14]:
data_test = df[[c for c in df.columns if c in [df.columns[0]]]] 
data_test

,ID_-0.510_29.290
2019_0,3.750994
2019_1,4.025176
2019_2,4.231381
2019_3,4.305286
2019_4,4.347317
...,...
2021_48,4.736880
2021_49,4.687898
2021_50,4.607418
2021_51,4.470275


In [18]:
look_ahead = 49
import numpy as np
def split(df):    
    X = df[:-look_ahead].T
    Y = df[-look_ahead:].T
    X = X.values[..., np.newaxis]
    Y = Y.values[..., np.newaxis]
    return X,Y  

In [19]:
x_test,y_test = split(data_test)
x_test.shape,y_test.shape

((1, 110, 1), (1, 49, 1))

In [20]:
x_test

array([[[3.7509942],
        [4.0251765],
        [4.231381 ],
        [4.3052855],
        [4.347317 ],
        [4.3108187],
        [4.2693343],
        [4.251361 ],
        [4.2819366],
        [4.3529334],
        [4.305424 ],
        [4.2373986],
        [4.3068933],
        [3.9083567],
        [4.515198 ],
        [4.32839  ],
        [4.418508 ],
        [4.4949617],
        [4.4770775],
        [4.520477 ],
        [4.4595714],
        [4.28684  ],
        [4.350335 ],
        [4.332584 ],
        [4.408427 ],
        [4.396392 ],
        [4.5531325],
        [4.4535413],
        [4.4382772],
        [4.4380097],
        [4.4041224],
        [4.6221538],
        [4.636855 ],
        [4.580318 ],
        [4.580013 ],
        [4.7120667],
        [4.6812696],
        [4.6184087],
        [4.707415 ],
        [4.4812164],
        [4.4857936],
        [4.4071417],
        [4.535888 ],
        [4.6048665],
        [4.4290185],
        [4.4828773],
        [4.5586553],
        [4.63

In [4]:
import tensorflow as tf

model_cnn = tf.keras.models.load_model('./data_preprocess/dillated_cnn')

model_cnn.summary()

Model: "dil_conv_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_16 (Conv1D)          multiple                  192       
                                                                 
 conv1d_17 (Conv1D)          multiple                  5152      
                                                                 
 conv1d_18 (Conv1D)          multiple                  5152      
                                                                 
 conv1d_19 (Conv1D)          multiple                  5152      
                                                                 
 dense_4 (Dense)             multiple                  33        
                                                                 
 lambda_4 (Lambda)           multiple                  0         
                                                                 
Total params: 15681 (61.25 KB)
Trainable params: 1

In [22]:
model_cnn(x_test)

<tf.Tensor: shape=(1, 49, 1), dtype=float32, numpy=
array([[[2.2492957],
        [2.2517018],
        [2.2336845],
        [2.1588907],
        [2.2694516],
        [2.2124805],
        [2.262331 ],
        [2.0788817],
        [1.9115394],
        [1.672035 ],
        [1.7210442],
        [1.7234318],
        [1.7917042],
        [1.8080726],
        [1.7812272],
        [1.886395 ],
        [1.8478702],
        [1.914744 ],
        [1.7385061],
        [1.9773761],
        [2.0038033],
        [2.1629775],
        [2.1313734],
        [2.1626534],
        [2.2250605],
        [2.227017 ],
        [2.2108307],
        [2.2993407],
        [2.4696822],
        [2.4033728],
        [2.504139 ],
        [2.5424213],
        [2.7113311],
        [2.537074 ],
        [2.4818182],
        [2.4342196],
        [2.4131968],
        [2.3544598],
        [2.3536315],
        [2.5159473],
        [2.4795995],
        [2.388144 ],
        [2.3525238],
        [2.4165916],
        [2.3185258],
   